# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [27]:
weatherdata = pd.read_csv("../WeatherPy/cities.csv") 

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
0,0,Laramie,75,US,1612575634,74,41.3114,-105.5911,25.268,6.17
1,1,Ushuaia,40,AR,1612575404,76,-54.8000,-68.3000,46.670,11.83
2,2,Warri,11,NG,1612575635,87,5.5167,5.7500,79.754,1.52
3,3,Hong Kong,0,HK,1612575605,77,22.2855,114.1577,69.278,2.01
4,4,Qaanaaq,100,GL,1612575636,77,77.4840,-69.3632,-3.838,3.36


In [28]:
weatherdata = {
    "City": weatherdata["City"],
    "Cloudiness": weatherdata["Cloudiness"],
    "Country": weatherdata["Country"],
    "Date": weatherdata["Date"], 
    "Humidity": weatherdata["Humidity"],
    "Lat": weatherdata["Lat"],
    "Long": weatherdata["Long"],
    "Max Temp": weatherdata["Max Temp"],
    "Wind Speed": weatherdata["Wind Speed"]
}

weather_df = pd.DataFrame(weatherdata)
weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
0,Laramie,75,US,1612575634,74,41.3114,-105.5911,25.268,6.17
1,Ushuaia,40,AR,1612575404,76,-54.8000,-68.3000,46.670,11.83
2,Warri,11,NG,1612575635,87,5.5167,5.7500,79.754,1.52
3,Hong Kong,0,HK,1612575605,77,22.2855,114.1577,69.278,2.01
4,Qaanaaq,100,GL,1612575636,77,77.4840,-69.3632,-3.838,3.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [29]:
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

locations = weather_df[["Lat", "Long"]].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=max(weather_df["Humidity"]), point_radius = 2)

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
narrowed_df = weather_df

# A max temperature lower than 80 degrees but higher than 70.
narrowed_df = narrowed_df.loc[(narrowed_df["Max Temp"] < 80) & (narrowed_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
narrowed_df = narrowed_df.loc[narrowed_df["Wind Speed"] < 10]

# Zero cloudiness.
narrowed_df = narrowed_df.loc[narrowed_df["Cloudiness"] == 0]

# Drop any rows with null values
narrowed_df = narrowed_df.dropna()

narrowed_df

,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
18,Necochea,0,AR,1612575639,50,-38.5473,-58.7368,71.276,1.34
25,Formosa,0,AR,1612575641,88,-26.1775,-58.1781,71.870,2.06
136,Durban,0,ZA,1612575433,94,-29.8579,31.0292,76.262,1.03
147,La Reforma,0,MX,1612575669,81,18.1000,-91.0167,71.816,1.04
168,Garoua,0,CM,1612575673,22,9.3000,13.4000,71.798,0.75
171,Garissa,0,KE,1612575507,78,-0.4536,39.6401,74.822,3.68
181,Pisco,0,PE,1612575677,83,-13.7000,-76.2167,71.870,4.63
189,Borborema,0,BR,1612575678,72,-21.6197,-49.0736,71.906,1.92
209,Vao,0,NC,1612575683,84,-22.6667,167.4833,78.422,4.92
270,Sawākin,0,SD,1612575698,52,19.1059,37.3321,70.520,3.07


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
